In [ ]:
import pandas as pd

import statsmodels.api as sm
import pandas as pd
import datetime
from pathlib import Path

pd.set_option("display.max_columns", None)
# !pip install statsmodels


# %config InlineBackend.figure_format = "svg"
# %config InlineBackend.print_figure_kwargs = {"dpi" : 300}
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


# from cadFace.vis import percentiles_plot
import sci_palettes

try:
    sci_palettes.register_cmap()
except:
    pass
import scienceplots
from pathlib import Path
import pandas as pd

# from ppp_aging.ppp_model import *

import json

plt.style.use(["nature", "no-latex"])
sns.set_context("paper", font_scale=1.5)
sns.set_palette("nejm")
from ppp_prediction.model import fit_best_model
from ppp_prediction.corr import generate_multipletests_result

## CVD list

CAD已有

Hypertension：服务器上有Hypertension.tab

Peripheral artery disease：服务器上有

Stroke：服务器上有Stroke.tab

Arrhythmia：服务器上有Atrial_fibrillation_or_flutter_v2.tab


添加疾病信息（这个后续会完善），因此这部分代码只需要跑一次

In [ ]:
import json

part2_dir = "/home/xutingfeng/ukb/project/ppp_prediction/result/part2"
mdoel_dir = f"{part2_dir}/models"
combination_dict_dir = f"{part2_dir}/dataset/combination_dict.json"
test_impute_dir = f"{part2_dir}/dataset/test_imputed.pkl"
train_impute_dir = f"{part2_dir}/dataset/train_imputed.pkl"

In [ ]:
test_impute = pd.read_pickle(test_impute_dir)
train_impute = pd.read_pickle(train_impute_dir)

In [ ]:
disease = pd.read_pickle(
    "/home/xutingfeng/ukb/ukbData/phenotypes/tf/ukb_common_disease_whole_life.pkl"
).reset_index(drop=False)
disease["eid"] = disease["eid"].astype(int)
disease

In [ ]:
train_impute = train_impute.merge(disease, on="eid", how="inner")
test_impute = test_impute.merge(disease, on="eid", how="inner")

In [ ]:
train_impute.to_pickle(train_impute_dir)
test_impute.to_pickle(test_impute_dir)

这个时候正常读取的数据是包含心血管疾病的数据的

## 提取json文件，运行代码

`parallel -q echo "./train_ml.py --train result/part1/0_all_cvd_prediction/dataset/train_imputed.pkl --test result/part1/0_all_cvd_prediction/dataset/test_imputed.pkl --output result/part1/0_all_cvd_prediction/models/{.} --json result/part1/0_all_cvd_prediction/dataset/{}" ::: $(cd result/part1/0_all_cvd_prediction/dataset; ls *.json) | parallel -j 4`

> 注意，请确保这里的train_impute 和后续的是一致的

In [ ]:
import json

part2_dir = "/home/xutingfeng/ukb/project/ppp_prediction/result/part2"
mdoel_dir = f"{part2_dir}/models"
combination_dict_dir = f"{part2_dir}/dataset/combination_dict.json"
test_impute_dir = f"{part2_dir}/dataset/test_imputed.pkl"
train_impute_dir = f"{part2_dir}/dataset/train_imputed.pkl"

combination_dict = json.load(open(combination_dict_dir, "r"))


# test_impute = pd.read_pickle(test_impute_dir)
# train_impute = pd.read_pickle(train_impute_dir)
disease = pd.read_pickle(
    "/home/xutingfeng/ukb/ukbData/phenotypes/tf/ukb_common_disease_whole_life.pkl"
).reset_index(drop=False)
disease["eid"] = disease["eid"].astype(int)
disease

In [ ]:
risk_factors = [
    "age",
    "sex",
    "ldl_a",
    "hdl_a",
    "tc_a",
    "tg_a",
    "sbp_a",
    "BMI",
    "smoking",
    "prevalent_diabetes",
]
AS = ["age", "sex"]
RF_without_AS = list(set(risk_factors) - set(AS))
RF = risk_factors
proteomics = combination_dict["proteomics"]["features"]

In [ ]:
disease_list = disease.columns[1:].tolist()
print(disease_list)


for disease_name in disease_list:
    disease_json = {
        "RF": {"features": RF, "label": disease_name},
        "RF_without_AS": {"features": RF_without_AS, "label": disease_name},
        "AS": {"features": AS, "label": disease_name},
        "proteomics": {"features": proteomics, "label": disease_name},
    }
    json.dump(
        disease_json,
        open(f"result/part1/0_all_cvd_prediction/dataset/{disease_name}.json", "w"),
    )

## 关联分析

In [ ]:
data = pd.read_pickle("result/part1/data.pkl")
if "hypertension" not in data.columns:
    data = data.merge(disease, on="eid", how="left")
data

In [ ]:
association_analysis_output_path = "result/part1/3_CVD_association_result"

fig_save_dir = "result/part1/3_CVD_association_result/figures"
Path(fig_save_dir).mkdir(parents=True, exist_ok=True)

In [31]:
# res_dict = {}
from ppp_prediction.corr import cal_corr

res = []
for disease_name in disease_list:
    cofounders = RF
    E = disease_name
    protein_cols = proteomics

    single_association_proteins_result_df = cal_corr(
        data, protein_cols, y=E, cofounders=cofounders, model_type="logit"
    )  # 使用原始没有imputated版本
    single_association_proteins_result_df["disease"] = E
    single_association_proteins_result_df.to_csv(
        f"{association_analysis_output_path}/{E}.csv", index=False
    )
    print(f"save {E}.csv")
    res.append(single_association_proteins_result_df)
res_df = pd.concat(res)

Optimization terminated successfully.
         Current function value: 0.238440
         Iterations 8


  0%|          | 6/2911 [00:03<31:38,  1.53it/s]

passed data have 51439 rows
used data have 33436 rows after dropna
Optimization terminated successfully.
         Current function value: 0.238694
         Iterations 8


  0%|          | 7/2911 [00:04<30:39,  1.58it/s]

passed data have 51439 rows
used data have 33510 rows after dropna
Optimization terminated successfully.
         Current function value: 0.238839
         Iterations 8


...

  9%|▉         | 274/2911 [01:17<08:05,  5.43it/s]

passed data have 51439 rows
used data have 33839 rows after dropna
Optimization terminated successfully.
         Current function value: 0.237063
         Iterations 8
passed data have 51439 rows
used data have 33839 rows after dropna
Optimization terminated successfully.
         Current function value: 0.237063
         Iterations 8


  9%|▉         | 275/2911 [01:17<10:51,  4.05it/s]

passed data have 51439 rows
used data have 33839 rows after dropna
Optimization terminated successfully.
         Current function value: 0.237060
         Iterations 8


  9%|▉         | 276/2911 [01:17<12:09,  3.61it/s]